<a href="https://colab.research.google.com/github/David-Cunha/multi-agent-video-generation-gemini/blob/main/CineAI_Roteiros_Inteligentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1️⃣ Instala bibliotecas
!pip install --upgrade google-generativeai --upgrade --quiet
!pip install google-generativeai google-genai --upgrade --quiet

In [ ]:
from google.colab import userdata
gemini_key = userdata.get('GEMINI_API_KEY')
gemini_key = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Sistema Multi-Agente para Google Colab - Interface Interativa
# Instalar dependências necessárias
!pip install google-generativeai google-genai ipywidgets pillow --upgrade --quiet

import os
import google.generativeai as genai
from google import genai as genai_new
from google.genai import types
import base64
import mimetypes
from pathlib import Path
import time
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import threading

# Para Google Colab
try:
    from google.colab import userdata
    COLAB_ENV = True
    print("🔍 Ambiente Google Colab detectado")
except ImportError:
    COLAB_ENV = False
    print("⚠️ Rodando fora do Google Colab")

# Configuração das APIs para Google Colab
def configurar_apis_colab():
    """Configura as chaves das APIs usando secrets do Google Colab"""
    if COLAB_ENV:
        try:
            gemini_key = userdata.get('GEMINI_API_KEY')

            if not gemini_key:
                return False, "❌ Chave GEMINI_API_KEY não encontrada nos secrets do Colab"

            # Usar apenas uma variável para evitar warning
            os.environ["GEMINI_API_KEY"] = gemini_key
            genai.configure(api_key=gemini_key)

            return True, "✅ API configurada usando secrets do Google Colab!"

        except Exception as e:
            return False, f"❌ Erro ao configurar API: {str(e)}"
    else:
        return False, "❌ Não está rodando no Google Colab"

# Função para criar pasta
def criar_pasta_projeto(nome_projeto, pasta_base="/content"):
    """Cria pasta para o projeto"""
    nome_limpo = "".join(c for c in nome_projeto if c.isalnum() or c in (' ', '-', '_')).rstrip()
    nome_limpo = nome_limpo.replace(' ', '_').lower()[:30]

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    pasta_destino = os.path.join(pasta_base, f"{nome_limpo}_{timestamp}")

    try:
        Path(pasta_destino).mkdir(parents=True, exist_ok=True)
        return pasta_destino, f"📁 Pasta criada: {pasta_destino}"
    except Exception as e:
        return pasta_base, f"❌ Erro ao criar pasta: {e}. Usando pasta padrão."

# Função para salvar arquivo
def save_binary_file(file_name, data, pasta_destino):
    """Salva arquivo binário"""
    caminho_completo = os.path.join(pasta_destino, file_name)
    try:
        with open(caminho_completo, "wb") as f:
            f.write(data)
        return caminho_completo, f"💾 Arquivo salvo: {file_name}"
    except Exception as e:
        return None, f"❌ Erro ao salvar {file_name}: {e}"

# Função para gerar texto
def gerar_texto(prompt, modelo="gemini-1.5-flash", temperatura=0.3):
    try:
        model = genai.GenerativeModel(
            model_name=modelo,
            generation_config=genai.types.GenerationConfig(
                temperature=temperatura,
                max_output_tokens=2048,
            )
        )
        response = model.generate_content(prompt)
        return True, response.text
    except Exception as e:
        return False, f"Erro ao gerar texto: {str(e)}"

# Função para gerar vídeo com VEO 2.0
def gerar_video_veo2(prompt_visual, pasta_destino, nome_projeto, callback=None):
    """Gera vídeo usando VEO 2.0 - modelo mais avançado"""
    try:
        if callback:
            callback("🎬 Iniciando geração com VEO 2.0...")
            callback("⏳ VEO 2.0 pode levar 2-5 minutos para gerar vídeos de alta qualidade")

        # Cliente específico para VEO 2.0
        client = genai_new.Client(
            http_options={"api_version": "v1beta"},
            api_key=os.environ.get("GEMINI_API_KEY"),
        )

        # Configuração específica do VEO 2.0
        video_config = types.GenerateVideosConfig(
            aspect_ratio="16:9",  # Formato widescreen
            number_of_videos=1,   # Gera 1 vídeo por vez
            duration_seconds=8,   # 8 segundos (máximo)
            person_generation="ALLOW_ALL",  # Permite pessoas no vídeo
        )

        # Otimiza o prompt para VEO 2.0
        prompt_otimizado = f"""
        {prompt_visual}

        Diretrizes para VEO 2.0:
        - Crie um vídeo cinematográfico profissional
        - Use transições suaves e naturais
        - Mantenha consistência visual ao longo do vídeo
        - Inclua movimento de câmera cinematográfico
        - Qualidade 4K com iluminação profissional
        - Narrative visual clara e envolvente
        """

        if callback:
            callback("🚀 Enviando prompt para VEO 2.0...")

        # Inicia a geração do vídeo
        operation = client.models.generate_videos(
            model="veo-2.0-generate-001",
            prompt=prompt_otimizado,
            config=video_config,
        )

        if callback:
            callback("⏳ VEO 2.0 está processando... Aguarde (isso pode levar alguns minutos)")

        # Aguarda a conclusão da geração
        check_count = 0
        while not operation.done:
            check_count += 1
            tempo_espera = min(10 + (check_count * 5), 30)  # Aumenta tempo gradualmente

            if callback:
                callback(f"🔄 Ainda processando... Checando novamente em {tempo_espera}s (tentativa {check_count})")

            time.sleep(tempo_espera)
            operation = client.operations.get(operation)

            # Timeout após 10 minutos
            if check_count > 20:
                return False, "⏰ Timeout: VEO 2.0 demorou mais que o esperado. Tente novamente mais tarde."

        if callback:
            callback("✅ VEO 2.0 concluiu a geração! Baixando vídeo...")

        # Processa o resultado
        result = operation.result
        if not result:
            return False, "❌ Erro: VEO 2.0 não retornou resultado válido"

        generated_videos = result.generated_videos
        if not generated_videos:
            return False, "❌ Nenhum vídeo foi gerado pelo VEO 2.0"

        arquivos_salvos = []

        # Salva os vídeos gerados
        for n, generated_video in enumerate(generated_videos):
            if callback:
                callback(f"💾 Baixando vídeo {n+1}/{len(generated_videos)}...")

            try:
                # Baixa o arquivo do servidor
                client.files.download(file=generated_video.video)

                # Define nome do arquivo
                nome_arquivo = f"{nome_projeto}_veo2_video_{n}.mp4"
                caminho_completo = os.path.join(pasta_destino, nome_arquivo)

                # Salva o vídeo
                generated_video.video.save(caminho_completo)
                arquivos_salvos.append(caminho_completo)

                if callback:
                    callback(f"✅ Vídeo salvo: {nome_arquivo}")

            except Exception as e:
                if callback:
                    callback(f"⚠️ Erro ao salvar vídeo {n}: {str(e)}")

        if arquivos_salvos:
            return True, arquivos_salvos
        else:
            return False, "❌ Nenhum vídeo foi salvo com sucesso"

    except Exception as e:
        error_msg = str(e)

        # Tratamento específico para diferentes tipos de erro
        if "429" in error_msg or "RESOURCE_EXHAUSTED" in error_msg:
            quota_msg = """
            ⚠️ Limite de uso da API VEO 2.0 atingido!

            📋 VEO 2.0 tem limites mais restritivos:
            • Aguarde mais tempo entre gerações
            • Verifique se tem créditos suficientes
            • VEO 2.0 requer plano pago para uso intensivo

            💡 Dica: Use o prompt visual salvo para gerar depois
            """
            return False, quota_msg
        elif "PERMISSION_DENIED" in error_msg:
            return False, "❌ VEO 2.0 não está disponível na sua região ou plano. Verifique as permissões da API."
        elif "INVALID_ARGUMENT" in error_msg:
            return False, "❌ Prompt muito longo ou contém conteúdo não permitido pelo VEO 2.0."
        else:
            return False, f"❌ Erro no VEO 2.0: {error_msg}"

# Agentes
def agente_pesquisador(topico):
    prompt = f"""
    Pesquise detalhadamente sobre: {topico}

    Inclua:
    - Definição e conceitos fundamentais
    - Estado atual e inovações recentes
    - Aplicações práticas no mundo real
    - Benefícios, desafios e limitações
    - Tendências futuras e potencial
    - Estatísticas e dados relevantes

    Seja didático e completo, pensando em educação do público geral.
    """
    return gerar_texto(prompt)

def agente_analista(texto):
    prompt = f"""
    Analise e organize para produção de vídeo:

    CONTEÚDO:
    {texto}

    ORGANIZE EM:
    1. CONCEITOS-CHAVE: Ideias principais
    2. STORYTELLING: Narrativa envolvente
    3. ELEMENTOS VISUAIS: O que mostrar
    4. MOMENTOS IMPACTANTES: Pontos de interesse
    5. MENSAGEM FINAL: Call-to-action

    Foque em engajamento e clareza visual.
    """
    return gerar_texto(prompt, temperatura=0.2)

def agente_roteirista(analise):
    prompt = f"""
    Crie roteiro profissional (3-4 min) baseado em:

    {analise}

    ESTRUTURA:
    - ABERTURA (0-30s): Hook + apresentação
    - DESENVOLVIMENTO (30s-3min): 4-5 cenas principais
    - CONCLUSÃO (3-4min): Resumo + call-to-action

    Para cada cena:
    - Duração, narração, descrição visual, transição

    Estilo cinematográfico e envolvente.
    """
    return gerar_texto(prompt, temperatura=0.2)

def agente_visual(roteiro):
    prompt = f"""
    Transforme em prompt visual unificado:

    ROTEIRO:
    {roteiro}

    CRIE PROMPT COM:
    - Estilo cinematográfico 4K
    - Movimentos de câmera suaves
    - Transições elegantes
    - Paleta harmoniosa
    - Narrativa visual clara
    - Elementos técnicos profissionais

    Um prompt coeso para vídeo de alta qualidade.
    """
    return gerar_texto(prompt, temperatura=0.1)

# Interface Interativa
class InterfaceVideoIA:
    def __init__(self):
        self.pasta_destino = "/content"
        self.topico = ""
        self.gerar_video_bool = True
        self.status_widget = None
        self.resultado = None

        self.criar_interface()

    def criar_interface(self):
        """Cria a interface interativa"""

        # Título
        display(HTML("""
        <div style='text-align: center; padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 10px; margin-bottom: 20px;'>
            <h1>🎬 CRIADOR DE VÍDEO COM IA</h1>
            <p>Sistema Multi-Agente para Geração Automática de Vídeos</p>
        </div>
        """))

        # Verificação de API
        sucesso, msg = configurar_apis_colab()
        if not sucesso:
            display(HTML(f"""
            <div style='background: #ffebee; border: 2px solid #f44336; padding: 15px; border-radius: 8px; margin: 10px 0;'>
                <h3>⚠️ Configuração Necessária</h3>
                <p>{msg}</p>
                <p><strong>Como configurar:</strong></p>
                <ol>
                    <li>Clique no ícone de chave (🗝️) na barra lateral esquerda</li>
                    <li>Adicione uma nova secret com nome: <code>GEMINI_API_KEY</code></li>
                    <li>Cole sua chave do Google AI Studio</li>
                    <li>Execute esta célula novamente</li>
                </ol>
            </div>
            """))
            return
        else:
            display(HTML(f"<div style='background: #e8f5e8; padding: 10px; border-radius: 5px; color: #2e7d2e;'>{msg}</div>"))

        # Seção 1: Configurações de Pasta
        pasta_section = self.criar_secao_pasta()
        display(pasta_section)

        # Seção 2: Seleção de Tópico
        topico_section = self.criar_secao_topico()
        display(topico_section)

        # Seção 3: Opções de Geração
        opcoes_section = self.criar_secao_opcoes()
        display(opcoes_section)

        # Seção 4: Botão de Execução
        execucao_section = self.criar_secao_execucao()
        display(execucao_section)

        # Status
        self.status_widget = widgets.Output()
        display(self.status_widget)

    def criar_secao_pasta(self):
        """Cria seção de configuração de pasta"""
        pasta_input = widgets.Text(
            value="/content",
            description="📁 Pasta:",
            style={'description_width': '100px'},
            layout=widgets.Layout(width='400px')
        )

        def on_pasta_change(change):
            self.pasta_destino = change['new']

        pasta_input.observe(on_pasta_change, names='value')

        pasta_info = widgets.HTML(
            value="<small>💡 Deixe em branco para usar /content (padrão do Colab)</small>"
        )

        return widgets.VBox([
            widgets.HTML("<h3>📂 Configuração de Pasta</h3>"),
            pasta_input,
            pasta_info
        ], layout=widgets.Layout(margin='10px 0'))

    def criar_secao_topico(self):
        """Cria seção de seleção de tópico"""

        topicos_predefinidos = [
            "Inteligência Artificial na Educação",
            "Sustentabilidade e Energia Renovável",
            "O Futuro do Trabalho Remoto",
            "Blockchain e Criptomoedas Explicadas",
            "Realidade Virtual na Medicina",
            "Machine Learning para Iniciantes",
            "Economia Circular e Meio Ambiente",
            "Biotecnologia e Medicina Personalizada",
            "Internet das Coisas (IoT) no Cotidiano",
            "Personalizado (digite abaixo)"
        ]

        topico_dropdown = widgets.Dropdown(
            options=topicos_predefinidos,
            value=topicos_predefinidos[0],
            description="🎯 Tópico:",
            style={'description_width': '100px'},
            layout=widgets.Layout(width='500px')
        )

        topico_custom = widgets.Text(
            placeholder="Digite seu tópico personalizado aqui...",
            description="✏️ Custom:",
            style={'description_width': '100px'},
            layout=widgets.Layout(width='500px'),
            disabled=True
        )

        def on_dropdown_change(change):
            if change['new'] == "Personalizado (digite abaixo)":
                topico_custom.disabled = False
                self.topico = ""
            else:
                topico_custom.disabled = True
                self.topico = change['new']

        def on_custom_change(change):
            if not topico_custom.disabled:
                self.topico = change['new']

        topico_dropdown.observe(on_dropdown_change, names='value')
        topico_custom.observe(on_custom_change, names='value')

        # Inicializar
        self.topico = topicos_predefinidos[0]

        return widgets.VBox([
            widgets.HTML("<h3>🎯 Seleção de Tópico</h3>"),
            topico_dropdown,
            topico_custom,
            widgets.HTML("<small>💡 Escolha um tópico pré-definido ou crie o seu próprio</small>")
        ], layout=widgets.Layout(margin='10px 0'))

    def criar_secao_opcoes(self):
        """Cria seção de opções de geração"""

        gerar_video_checkbox = widgets.Checkbox(
            value=True,
            description="🎬 Gerar vídeo com VEO 2.0 (recomendado)",
            style={'description_width': 'initial'}
        )

        def on_video_change(change):
            self.gerar_video_bool = change['new']

        gerar_video_checkbox.observe(on_video_change, names='value')

        veo_info = widgets.HTML(
            value="""
            <div style='background: #f0f7ff; padding: 10px; border-radius: 5px; margin: 5px 0;'>
                <strong>🚀 VEO 2.0 Features:</strong><br>
                • Qualidade cinematográfica profissional<br>
                • Vídeos de até 8 segundos em 4K<br>
                • Movimentos de câmera realistas<br>
                • Pessoas e objetos consistentes<br>
                • Tempo de geração: 2-5 minutos
            </div>
            """
        )

        return widgets.VBox([
            widgets.HTML("<h3>⚙️ Opções de Geração</h3>"),
            gerar_video_checkbox,
            veo_info,
            widgets.HTML("<small>⚠️ VEO 2.0 requer créditos na conta Google AI</small>")
        ], layout=widgets.Layout(margin='10px 0'))

    def criar_secao_execucao(self):
        """Cria seção com botão de execução"""

        botao_executar = widgets.Button(
            description="🚀 CRIAR VÍDEO",
            button_style='success',
            layout=widgets.Layout(width='200px', height='50px'),
            style={'font_weight': 'bold'}
        )

        def on_click(b):
            if not self.topico.strip():
                self.mostrar_status("❌ Por favor, selecione ou digite um tópico!", 'error')
                return

            self.executar_pipeline()

        botao_executar.on_click(on_click)

        return widgets.VBox([
            widgets.HTML("<h3>🚀 Execução</h3>"),
            botao_executar,
            widgets.HTML("<small>🕒 Tempo estimado: 3-10 minutos (dependendo da geração de vídeo)</small>")
        ], layout=widgets.Layout(margin='20px 0'))

    def mostrar_status(self, mensagem, tipo='info'):
        """Mostra status na interface"""
        cores = {
            'info': '#e3f2fd',
            'success': '#e8f5e8',
            'error': '#ffebee',
            'warning': '#fff3e0'
        }

        cor_borda = {
            'info': '#2196f3',
            'success': '#4caf50',
            'error': '#f44336',
            'warning': '#ff9800'
        }

        cor_texto = {
            'info': '#1565c0',
            'success': '#2e7d32',
            'error': '#c62828',
            'warning': '#f57c00'
        }

        with self.status_widget:
            display(HTML(f"""
            <div style='background: {cores.get(tipo, cores["info"])};
                        border-left: 4px solid {cor_borda.get(tipo, cor_borda["info"])};
                        padding: 10px; margin: 5px 0; border-radius: 4px;
                        color: {cor_texto.get(tipo, cor_texto["info"])};
                        font-weight: 500;'>
                {mensagem}
            </div>
            """))

    def executar_pipeline(self):
        """Executa o pipeline completo"""

        with self.status_widget:
            clear_output(wait=True)

        self.mostrar_status("🚀 Iniciando sistema multi-agente...", 'info')

        # Criar pasta
        pasta_final, msg_pasta = criar_pasta_projeto(self.topico, self.pasta_destino)
        self.mostrar_status(msg_pasta, 'info')

        nome_projeto = self.topico.replace(" ", "_").lower()[:20]
        resultados = {}

        try:
            # Etapa 1: Pesquisa
            self.mostrar_status("🔎 [1/4] Executando agente pesquisador...", 'info')
            sucesso, pesquisa = agente_pesquisador(self.topico)
            if not sucesso:
                self.mostrar_status(f"❌ Erro na pesquisa: {pesquisa}", 'error')
                return

            with open(f"{pasta_final}/01_pesquisa.txt", 'w', encoding='utf-8') as f:
                f.write(pesquisa)
            resultados['pesquisa'] = pesquisa
            self.mostrar_status("✅ Pesquisa concluída!", 'success')

            # Etapa 2: Análise
            self.mostrar_status("📊 [2/4] Executando agente analista...", 'info')
            sucesso, analise = agente_analista(pesquisa)
            if not sucesso:
                self.mostrar_status(f"❌ Erro na análise: {analise}", 'error')
                return

            with open(f"{pasta_final}/02_analise.txt", 'w', encoding='utf-8') as f:
                f.write(analise)
            resultados['analise'] = analise
            self.mostrar_status("✅ Análise concluída!", 'success')

            # Etapa 3: Roteiro
            self.mostrar_status("🎬 [3/4] Executando agente roteirista...", 'info')
            sucesso, roteiro = agente_roteirista(analise)
            if not sucesso:
                self.mostrar_status(f"❌ Erro no roteiro: {roteiro}", 'error')
                return

            with open(f"{pasta_final}/03_roteiro.txt", 'w', encoding='utf-8') as f:
                f.write(roteiro)
            resultados['roteiro'] = roteiro
            self.mostrar_status("✅ Roteiro concluído!", 'success')

            # Etapa 4: Prompt Visual
            self.mostrar_status("🎨 [4/4] Executando agente visual...", 'info')
            sucesso, prompt_visual = agente_visual(roteiro)
            if not sucesso:
                self.mostrar_status(f"❌ Erro no prompt visual: {prompt_visual}", 'error')
                return

            with open(f"{pasta_final}/04_prompt_visual.txt", 'w', encoding='utf-8') as f:
                f.write(prompt_visual)
            resultados['prompt_visual'] = prompt_visual
            self.mostrar_status("✅ Prompt visual concluído!", 'success')

            # Etapa 5: Geração de Vídeo com VEO 2.0
            if self.gerar_video_bool:
                self.mostrar_status("🎬 [EXTRA] Iniciando geração com VEO 2.0...", 'warning')
                self.mostrar_status("🚀 VEO 2.0 é o modelo mais avançado do Google para vídeos!", 'info')

                def callback_video(msg):
                    self.mostrar_status(msg, 'info')

                sucesso, resultado_video = gerar_video_veo2(prompt_visual, pasta_final, nome_projeto, callback_video)

                if sucesso and resultado_video:
                    self.mostrar_status(f"🎉 {len(resultado_video)} vídeo(s) VEO 2.0 gerado(s) com sucesso!", 'success')
                    resultados['arquivos_video'] = resultado_video

                    # Mostra informações dos vídeos gerados
                    for i, video in enumerate(resultado_video):
                        tamanho = os.path.getsize(video) / (1024 * 1024)  # MB
                        nome = os.path.basename(video)
                        self.mostrar_status(f"🎬 {nome} - {tamanho:.1f} MB - Formato: 4K 16:9", 'success')

                elif sucesso and not resultado_video:
                    self.mostrar_status("⚠️ VEO 2.0 processou mas nenhum vídeo foi salvo. Verifique os logs acima.", 'warning')
                else:
                    # Erro na geração - mostra mensagem formatada
                    self.mostrar_status(resultado_video, 'error')
                    self.mostrar_status("💡 Você ainda tem o roteiro completo! Pode tentar gerar o vídeo mais tarde usando o prompt_visual.txt", 'info')
            else:
                self.mostrar_status("⏭️ Geração de vídeo desabilitada (por escolha do usuário)", 'info')

            # Resumo final
            self.mostrar_status("🎉 PIPELINE CONCLUÍDO COM SUCESSO!", 'success')
            self.mostrar_status(f"📂 Pasta do projeto: <code>{pasta_final}</code>", 'info')

            # Listar arquivos
            arquivos = list(Path(pasta_final).glob('*'))
            if arquivos:
                lista_arquivos = ""
                for arq in sorted(arquivos):
                    tamanho = arq.stat().st_size / 1024  # KB
                    lista_arquivos += f"<div style='margin: 2px 0; padding: 5px; background: rgba(255,255,255,0.1); border-radius: 3px;'>📄 <strong>{arq.name}</strong> ({tamanho:.1f} KB)</div>"

                self.mostrar_status(f"📋 <strong>Arquivos criados:</strong><br>{lista_arquivos}", 'info')

                # Mostrar comando para listar arquivos
                self.mostrar_status(f"💻 Para ver os arquivos no Colab: <code>!ls -la {pasta_final}</code>", 'info')

            self.resultado = resultados

        except Exception as e:
            self.mostrar_status(f"❌ Erro durante execução: {str(e)}", 'error')

# Criar e exibir interface
interface = InterfaceVideoIA()